## Changes this notebook makes to ArtsEngagementMerged.sav
Saves the resulting data to as an Rdata object data/artsengagement.Rda
- Converts labelled columns into factors
- Converts blank free responses to NA
- Removes columns with no responses
- Sets checkbox columns to include No as well as NA (ie. set sr_theatre_leadership to No if student marked sr_theatre_leadership)
- Replaces "Did not respond..." & "Saw survey, did not respond" values with NA


- Adds the following columns:
    - Number of surveys taken (surveys_taken)
    - Art engagement levels (engagement_level)
    - Major categorization (field (most recent from each student))
    - Whether or not a student participated or lead any art activity in a given year, over their entire college experience
    - Demographics:
        - sex
        - ethnic_group
        - major
        - college
        - college2 (acronyms, but somehow there are different from the other college)
        - GPA
        - first_term (first term attended)
        - completion_term
- Removes uniqname
        
### Assumptions this notebook makes
the following files are in the directory data in the current path:
    - ArtsEngagementMerged.sav
    - ArtsEngagementMerged_V2_no_uniqname.sav
    - unique_name+key_artsengagement_v2.csv (but not needed)

In [ ]:
library(tidyverse)
library(magrittr)
library(haven)

In [2]:
df <- read_sav('data/ArtsEngagementMerged.sav')

In [3]:
source('select_helpers.R')
source('add_cols.R')
source('recode_data.R')

In [4]:
# Legacy code needed to add new columns
temp <- df
temp <- recode_data(temp)
temp <- add_all_cols(temp)

df <- bind_cols(df, select(temp, (ncol(temp)-9):ncol(temp)))

adding participated columns (bool)...
categorizing major columns (factor)...
adding art engagement levels (numeric)...
adding number of surveys taken (numeric)...

new columns:
participated
fr_participated
so_participated
jr_participated
sr_participated
college_participated
hs_participated
field
engagement_level
surveys_taken


# Converting

Recode all -99s and -98s to NA (Did not respond to this survey, and Saw survey did not respond)

In [5]:
df[df==-99] <- NA
df[df==-98] <- NA

Labelled to factor

In [6]:
df[df %>% sapply(is.labelled)] %<>% as_factor

Removing blank free responses

In [7]:
df[df %>% sapply(is.character)] %<>% sapply(zap_empty)

df$definition[is.na(df$definition)] %>% head

[1] NA NA NA NA NA NA

In [8]:
#cols that have 2 classes all end with 'Date'
df %>% sapply(class) %>% sapply(function(x)(x[[1]])) %>% table

.
character    factor   integer   logical   numeric   POSIXct 
       72       973         1         7       809        12 

Removing extraneous columns (those with no responses)

In [9]:
to_drop <- df[df %>% sapply(function(x){return(length(unique(x))==1)})] %>% names
to_drop %>% print
df[to_drop] <- NULL

[1] "fl_intro2" "intro2"   


Finding numeric columns with more than 1 type of response

In [10]:
num <- df[df %>% sapply(is.numeric)] %>% names
uni <- df[df %>% sapply(function(x){return(length(unique(x))==2)})] %>% names

num[!(num %in% uni)]

[1] "f1_Finished"      "f2_Finished"      "fl_Finished"      "so_Finished"     
[5] "jr_Finished"      "sr_Finished"      "engagement_level" "surveys_taken"

We want to deal with these columns separately

Set checkbox columns to include No as well as NA (ie. set sr_theatre_leadership to No if student marked sr_theatre_none)

In [11]:
df %>% select(matches('sr_theatre_(none|attendance|participation|leadership)')) %>% 
        group_by(sr_theatre_none, sr_theatre_attendance, sr_theatre_participation, sr_theatre_leadership) %>% 
        summarize(count=n()) %>% arrange(desc(count))

sr_theatre_none,sr_theatre_attendance,sr_theatre_participation,sr_theatre_leadership,count
NA,NA,NA,NA,3546
1,NA,NA,NA,608
NA,1,NA,NA,545
NA,NA,1,NA,54
NA,1,1,NA,36
NA,NA,NA,1,17
NA,1,1,1,12
1,1,NA,NA,6
1,1,1,1,1
1,NA,1,NA,1


We can't combine these columns

In [12]:
df_backup <- df #important for later

In [117]:
year <- c('(hs)', '(fl|f1|f2)', '(so|sophlate)', '(jr|jl)', '(sr|sl)')
art_categories <- c("animation","architecture","comedy","comics",
               "crafts","cwriting","dance","film","gdesign",
               "multimedia","muscomp","musperf","theatre","visart","other")
motivOrPartic <- c("(money|other|credit|fun)", ("(none|attendance|participation|leadership)"))

temp <- df[df %>% sapply(function(x){return(length(unique(x))==2 & is.numeric(x))})]
for(yr_idx in seq(year)) {
    for(art_idx in seq(art_categories)) {
        for(mop_idx in seq(motivOrPartic)) {
            pattern <- paste(year[yr_idx], "_", art_categories[art_idx], "_", motivOrPartic[mop_idx], sep="")
            datum <- temp[temp %>% names %>% grepl(pattern = pattern, perl=TRUE)]
            if(length(datum)!=0) {
                filled <- apply(datum, 1, function(x)sum(!is.na(x)))
                datum[sapply(filled, function(x)x!=0),] %<>% is.na()
                datum[sapply(filled, function(x)x!=0),] <- !datum[sapply(filled, function(x)x!=0),]
                temp[temp %>% names %>% grepl(pattern = pattern, perl=TRUE)] <- datum %>% sapply(as.logical)
            }
        }
    }
}
df[df %>% sapply(function(x){return(length(unique(x))==2 & is.numeric(x))})] <- temp

In [118]:
categories <- df[df %>% sapply(function(x){return(length(unique(x))==2 & is.numeric(x))})] %>% names
categories <- regexpr("_\\w+_", names(temp), perl=TRUE) %>% regmatches(x = names(temp)) %>% unique %>%
                lapply(function(x){return(substr(x, 2, nchar(x)-1))})

temp <- df[df %>% sapply(function(x){return(length(unique(x))==2 & is.numeric(x))})]
for(yr_idx in seq(year)) {
    for(category in seq(categories)) {
        pattern <- paste(year[yr_idx], "_", categories[category], "_\\w+", sep="")
        datum <- temp[temp %>% names %>% grepl(pattern = pattern, perl=TRUE)]
        if(length(datum)!=0) {
            filled <- apply(datum, 1, function(x)sum(!is.na(x)))
            datum[sapply(filled, function(x)x!=0),] %<>% is.na()
            datum[sapply(filled, function(x)x!=0),] <- !datum[sapply(filled, function(x)x!=0),]
            temp[temp %>% names %>% grepl(pattern = pattern, perl=TRUE)] <- datum %>% sapply(as.logical)
        }
    }
  }
df[df %>% sapply(function(x){return(length(unique(x))==2 & is.numeric(x))})] <- temp

## Deal with remaining numeric columns

In [119]:
df %>% sapply(class) %>% sapply(function(x)(x[[1]])) %>% table

.
character    factor   integer   logical   numeric   POSIXct 
       72       973         1       807         7        12 

In [120]:
df[df %>% sapply(is.numeric)] %>% names

[1] "f1_Finished"      "f2_Finished"      "fl_Finished"      "so_Finished"     
[5] "jr_Finished"      "sr_Finished"      "engagement_level" "surveys_taken"

In [121]:
finished_cols <- df %>% select(ends_with('Finished')) %>% names
df_backup[[finished_cols[1]]] %>% attributes
df_backup[[finished_cols[1]]] %>% table

$label
[1] "Finished"

$format.spss
[1] "F1.0"

$display_width
[1] 0

.
   0    1 
 268 3589 

In [122]:
for(i in seq(finished_cols)) {
    attribs <- df[[finished_cols[i]]] %>% attributes
    df[[finished_cols[i]]] %<>% factor(labels=c('No','Yes'))
    attr(df[[finished_cols[i]]], 'label') <- attribs$label
}

In [123]:
df[[finished_cols[1]]] %>% summary

No  Yes NA's 
 268 3589  971

## Turn resulting logical columns into factors

In [124]:
df %>% sapply(class) %>% sapply(function(x)(x[[1]])) %>% table

df[df %>% sapply(is.logical)] %>% sapply(function(x)(is.null(attributes(x)))) %>% length

.
character    factor   integer   logical   numeric   POSIXct 
       72       979         1       807         1        12 

[1] 807

All attributes of the logical columns are null

In [125]:
logical_cols <- df[df %>% sapply(is.logical)] %>% names

for(i in seq(logical_cols)) {
    if((df[[logical_cols[i]]] %>% table %>% names %>% .[1]) == 'FALSE') {# To make sure No-False and Yes-True line up
        df[[logical_cols[i]]] %<>% factor(labels=c('No','Yes'))
        attr(df[[logical_cols[i]]], 'label') <- df_backup[[logical_cols[i]]] %>% attributes %>% .$label
    }
}

In [126]:
df %>% sapply(class) %>% sapply(function(x)(x[[1]])) %>% table

.
character    factor   integer   numeric   POSIXct 
       72      1786         1         1        12 

## Replace "Did not respond..." & "Saw survey, did not respond"  values with NA

In [127]:
cols <- df[df %>% sapply(is.factor)] %>% names
didnotCols <- c()
for(i in seq(cols)) {
    if (df[[cols[i]]] %>% table %>% names %>% .[1] == 'Did not respond to this survey') {
        didnotCols %<>% c(cols[i])
        df[df[cols[i]]=='Did not respond to this survey' & !is.na(df[cols[i]]), cols[i]] <- NA
        df[df[cols[i]]=='Saw survey, did not respond to question' & !is.na(df[cols[i]]), cols[i]] <- NA
        df[cols[i]] %<>% droplevels
    }
}

In [128]:
didnotCols %>% length

[1] 268

In [129]:
cols[(cols %in% didnotCols)]

[1] "f1_surveyterm"                   "f2_surveyterm"                  
  [3] "fl_surveyterm"                   "so_surveyterm"                  
  [5] "sophlate_surveyterm"             "jr_surveyterm"                  
  [7] "jl_surveyterm"                   "sr_surveyterm"                  
  [9] "sl_surveyterm"                   "hs_animation_attendance"        
 [11] "hs_animation_participation"      "hs_animation_leadership"        
 [13] "hs_animation_none"               "hs_architecture_attendance"     
 [15] "hs_architecture_participation"   "hs_architecture_leadership"     
 [17] "hs_architecture_none"            "hs_comedy_attendance"           
 [19] "hs_comedy_participation"         "hs_comedy_leadership"           
 [21] "hs_comedy_none"                  "hs_comics_attendance"           
 [23] "hs_comics_participation"         "hs_comics_leadership"           
 [25] "hs_comics_none"                  "hs_crafts_attendance"           
 [27] "hs_crafts_participation"         "hs_crafts_leadership"           
 [29] "hs_crafts_none"                  "hs_cwriting_attendance"         
 [31] "hs_cwriting_participation"       "hs_cwriting_leadership"         
 [33] "hs_cwriting_none"                "hs_dance_attendance"            
 [35] "hs_dance_participation"          "hs_dance_leadership"            
 [37] "hs_dance_none"                   "hs_film_attendance"             
 [39] "hs_film_participation"           "hs_film_leadership"             
 [41] "hs_film_none"                    "hs_gdesign_attendance"          
 [43] "hs_gdesign_participation"        "hs_gdesign_leadership"          
 [45] "hs_gdesign_none"                 "hs_multimedia_attendance"       
 [47] "hs_multimedia_participation"     "hs_multimedia_leadership"       
 [49] "hs_multimedia_none"              "hs_muscomp_attendance"          
 [51] "hs_muscomp_participation"        "hs_muscomp_leadership"          
 [53] "hs_muscomp_none"                 "hs_musperf_attendance"          
 [55] "hs_musperf_participation"        "hs_musperf_leadership"          
 [57] "hs_musperf_none"                 "hs_theatre_attendance"          
 [59] "hs_theatre_participation"        "hs_theatre_leadership"          
 [61] "hs_theatre_none"                 "hs_visart_attendance"           
 [63] "hs_visart_participation"         "hs_visart_leadership"           
 [65] "hs_visart_none"                  "hs_other_attendance"            
 [67] "hs_other_participation"          "hs_other_leadership"            
 [69] "hs_other_none"                   "hs_animation_credit"            
 [71] "hs_animation_money"              "hs_animation_fun"               
 [73] "hs_animation_other"              "hs_architecture_credit"         
 [75] "hs_architecture_money"           "hs_architecture_fun"            
 [77] "hs_architecture_other"           "hs_comedy_credit"               
 [79] "hs_comedy_money"                 "hs_comedy_fun"                  
 [81] "hs_comedy_other"                 "hs_comics_credit"               
 [83] "hs_comics_money"                 "hs_comics_fun"                  
 [85] "hs_comics_other"                 "hs_crafts_credit"               
 [87] "hs_crafts_money"                 "hs_crafts_fun"                  
 [89] "hs_crafts_other"                 "hs_cwriting_credit"             
 [91] "hs_cwriting_money"               "hs_cwriting_fun"                
 [93] "hs_cwriting_other"               "hs_dance_credit"                
 [95] "hs_dance_money"                  "hs_dance_fun"                   
 [97] "hs_dance_other"                  "hs_film_credit"                 
 [99] "hs_film_money"                   "hs_film_fun"                    
[101] "hs_film_other"                   "hs_gdesign_credit"              
[103] "hs_gdesign_money"                "hs_gdesign_fun"                 
[105] "hs_gdesign_other"                "hs_multimedia_credit"           
[107] "hs_multimedia_money"             "hs_multimedia_fun"              
[109] 

## Change responseSet variables to factors

In [ ]:
df %<>% mutate_at(vars(ends_with('Set')), as.factor)

# Import Demographic Data

In [ ]:
demog <- read_sav(file = 'data/ArtsEngagementMerged_V2_no_uniqname.sav')

In [6]:
key <- read.csv(file = 'data/unique_name+key_artsengagement_v2.csv')
# not inherently needed

In [56]:
data.frame(head(key),head(demog %>% select(key)))
data.frame(tail(key),tail(demog %>% select(key)))

X,temp,key,key.1
1,aabdunna,WKRVG3695F,WKRVG3695F
2,aabeauch,VKBLK5812B,VKBLK5812B
3,aabichir,JGUST9616X,JGUST9616X
4,aacorona,GWIFC7315D,GWIFC7315D
5,aafonso,OKCUD3419W,OKCUD3419W
6,aaforsg,TOONA3838W,TOONA3838W


,X,temp,key,key.1
4823,4823,zousley,EDCPH0173Z,EDCPH0173Z
4824,4824,zpayne,YJIBJ1967N,YJIBJ1967N
4825,4825,zvandyke,AAHTC4583E,AAHTC4583E
4826,4826,zweglarz,MADAD5099B,MADAD5099B
4827,4827,zwenchen,PGCOD0004W,PGCOD0004W
4828,4828,zzent,JNNTL5446R,JNNTL5446R


demog data is already aligned with df (alphabetized by uniqname)

In [16]:
demog[!(names(demog) %in% names(df))] %>% names

[1] "fl_intro2"                      "intro2"                        
 [3] "key"                            "SEX_DESCRSHORT"                
 [5] "ETHNIC_GROUP"                   "ETHNIC_GROUP_DESCRSHORT"       
 [7] "ACAD_PLAN"                      "ACAD_PLAN_DESCR"               
 [9] "ACAD_GROUP_DESCR"               "CUM_GPA"                       
[11] "FIRST_TERM_ATTENDED"            "FIRST_TERM_ATTENDED_DESCRSHORT"
[13] "COMPLETION_TERM"                "COMPLETION_TERM_DESCR"         
[15] "ACAD_CAREER"

In [21]:
# sex
demog$SEX_DESCRSHORT[demog$SEX_DESCRSHORT == 'NA'] <- NA
df$sex <- demog$SEX_DESCRSHORT %>% zap_missing %>% zap_empty %>% zap_formats %>% as_factor

In [26]:
# ethnic_group
demog$ETHNIC_GROUP_DESCRSHORT[demog$ETHNIC_GROUP_DESCRSHORT == 'NA'] <- NA
df$ethnic_group <- demog$ETHNIC_GROUP_DESCRSHORT %>% zap_missing %>% zap_empty %>% zap_formats %>% as_factor

In [33]:
# major
demog$ACAD_PLAN_DESCR[demog$ACAD_PLAN_DESCR == 'NA'] <- NA
df$major <- demog$ACAD_PLAN_DESCR %>% zap_missing %>% zap_empty %>% zap_formats %>% as_factor

In [35]:
# college
demog$ACAD_GROUP_DESCR[demog$ACAD_GROUP_DESCR == 'NA'] <- NA
df$college <- demog$ACAD_GROUP_DESCR %>% zap_missing %>% zap_empty %>% zap_formats %>% as_factor

In [39]:
# GPA
demog$CUM_GPA[demog$CUM_GPA == 'NA'] <- NA
df$GPA <- demog$CUM_GPA %>% zap_missing %>% zap_empty %>% zap_formats %>% as.numeric
df$GPA[df$GPA == 0 & !is.na(df$GPA)] <- NA

In [48]:
# first_term
demog$FIRST_TERM_ATTENDED_DESCRSHORT[demog$FIRST_TERM_ATTENDED_DESCRSHORT == 'NA'] <- NA
df$first_term <- demog$FIRST_TERM_ATTENDED_DESCRSHORT %>% zap_missing %>% zap_empty %>% zap_formats %>% as_factor

In [49]:
# completion_term
demog$COMPLETION_TERM_DESCR[demog$COMPLETION_TERM_DESCR == 'NA'] <- NA
df$completion_term <- demog$COMPLETION_TERM_DESCR %>% zap_missing %>% zap_empty %>% zap_formats %>% as_factor

In [50]:
# college2
demog$ACAD_CAREER[demog$ACAD_CAREER == 'NA'] <- NA
df$college2 <- demog$ACAD_CAREER %>% zap_missing %>% zap_empty %>% zap_formats %>% as_factor

### Replace uniqname with key

In [ ]:
df <- bind_cols(select(key, key), select(df, -uniqname))

# STM Data

to do

---

In [53]:
library(broom)

In [54]:
zeroLabelCols <- df[df %>% sapply(is.factor)] %>% 
                        sapply(function(dat)(dat %>% summary %>% tidy %>% arrange(x) %>% slice(1) %>% .$x == 0))

In [55]:
df[df %>% sapply(is.factor)] %>% .[zeroLabelCols] %>% names

[1] "fl_hstype"               "so_highestdegreeplanned"
[3] "jr_highestdegreeplanned" "sr_highestdegreeplanned"

**Be aware of these columns, as some packages will fail with factors that have levels with 0 occurrences**, or even less than 5

# Save

In [147]:
save(df, file='data/artsengagement.Rda')

In [12]:
load('data/artsengagement.Rda')

# Optional: merge engagement columns
eg. sr_comedy_none, sr_comedy_participation, etc, becomes a factor called sr_comedy with levels none, participation, etc

In [ ]:
year <- c('(hs)', '(fl|f1|f2)', '(so|sophlate)', '(jr|jl)', '(sr|sl)')
year_names <- c('hs', 'f2', 'so', 'jr', 'sr')

categories <- c('animation', 'architecture', 'comedy', 'comics', 'crafts', 'cwriting', 'dance', 'film',
                'gdesign', 'multimedia', 'muscomp', 'musperf', 'theatre', 'visart', 'other', 'arts', 'athletics',
                'commservice', 'honors', 'clubs', 'govt', 'professional')

part_lvl <- c('none', 'attendance', 'participation', 'leadership')

merged_pattern <- paste("\\w+_(", paste(categories, collapse='|'), ")_(none|attendance|participation|leadership)", sep="")
temp <- df[df %>% names %>% grepl(pattern = merged_pattern, perl=TRUE)]

merged.df <- data_frame(junk = rep(NA, nrow(temp)))
for(yr_idx in seq(year)) {
    for(cat_idx in seq(categories)) {
        pattern <- paste(year[yr_idx], "_", categories[cat_idx], 
                         "_(none|attendance|participation|leadership)", sep="")
        datum <- temp[temp %>% names %>% grepl(pattern = pattern, perl=TRUE)]
        if(length(datum)!=0) {
            merged <- c(rep(NA, nrow(datum)))
            for(part_idx in seq(part_lvl)) {
                datumia <- datum %>% select(ends_with(part_lvl[part_idx])) %>% as.matrix
                for(row in seq(nrow(datum))) {
                    if(!is.na(datumia[row]) & datumia[row] == 'Yes') {
                        merged[row] <- part_lvl[part_idx]
                    }
                }
            }
            if(length(unique(merged)) > 1) {
                merged.df %<>% mutate(newCol = merged)
                merged.df$newCol %<>% as.factor
                attr(merged.df$newCol, 'label') <- datum %>% select(ends_with(part_lvl[1])) %>% .[[1]] %>%
                                                             attributes %>% .$label %>% substr(., 1,nchar(.)-5)
                names(merged.df)[ncol(merged.df)] <- paste(year_names[yr_idx], '_', categories[cat_idx], sep='')
            }
        }
    }
}
merged.df %>% select(-junk)

df <- df[!(df %>% names %>% grepl(pattern = merged_pattern, perl=TRUE))]
df <- bind_cols(df[1:22], merged.df, df[23:ncol(df)])